In [2]:
import codecs
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import sys, random
import pandas as pd
from konlpy.tag import Twitter
import re

Using TensorFlow backend.


In [3]:
fp = codecs.open('투캅스(샘플).txt', 'r', 'utf-8')
lines = fp.readlines()
fp.close()


In [4]:
# with open('투캅스(샘플).txt', 'rb') as fp2:  # codecs를 안쓰는 법???
#     lines2 = fp2.read()
# lines2 = lines2.decode('utf-8')
# # print(lines)
# lines2 = ''.join(lines2).split('\n')
# print(lines2)

In [5]:
results = []

for line in lines:
    cleantext = BeautifulSoup(line, 'lxml').text # 전에는 lxml이 아닌 html.parser 사용
    # lxml을 사용하면 html.parser보다 빠르지만 따로 설치가 필요, parser종류 중 하나
    # lxml은 c언어 라이브러리? 더 유연함.
    cleantext = re.sub('[\r\n!@#$]', '', cleantext)
    results.append(cleantext)

all_text = ' '.join(results)
text = all_text
print('훈련 데이터', len(text))

훈련 데이터 67523


In [6]:
text_split = text.split(' ')
chars = sorted(list(set(text_split))) # 단어 별 중복 제거하고 글자 순으로 나열
print('단어의 종류', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 5
step = 1
sentences = []
next_chars = []

for i in range(0, len(text_split) - maxlen, step): # 전의 질병패턴같이 중복해서 학습
    sentences.append(' '.join(text_split[i: i + maxlen]))
    next_chars.append(text_split[i + maxlen])
    
# sentences == 5단어로 이뤄진 문장들의 list
print('학습할 문장의 수:', len(sentences))
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence.split(' ')):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print(X.shape, y.shape)

단어의 종류 7696
학습할 문장의 수: 15229
(15229, 5, 7696) (15229, 7696)


In [7]:
print('lstm 모델을 만들어 본다...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)))) # 128 메모리 셀을 가진 LSTM 레이어 생성
model.add(Dense(len(chars)))  # 완전 연결 레이어(Dense) 레이어 == 전결합층이라 불림
# dense : 입력과 출력을 모두 연결해줌.
model.add(Activation('softmax'))
# softmax를 사용하면 0에서 1사이이며, 모든 확률의 합은 1이다.
# 출력층에서 확률표현을 위해 사용됨.
optimizer = RMSprop(lr=0.015)
# RNN의 옵티마이저로 많이 사용됨.
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
# crossentropy == log씌운 미분식. loss function.

def sample(preds, temp=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
#     print(preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

lstm 모델을 만들어 본다...


In [8]:
generated_all = []
for iteration in range(1, 8):
    print()
    print('반복번호 = ', iteration)
    model.fit(X, y, batch_size=128, epochs=1)
    
    for diversity in [1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6]:
        print()
        print('추출옵션 : ', diversity)
        generated =''
        sentence ='범인을 빨리 잡아야지.'
        generated += sentence
        print('질문시작패턴 = "' + sentence + '"')
        sys.stdout.write(generated)
        
        for i in range(1):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence.split(' ')):
                x[0, t, char_indices[char]] = 1
            
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            
            generated += " " + next_char
            sentence = ' '.join(sentence.split(' ')[1:]) + ' ' + next_char
            sys.stdout.write(' ' + next_char)
            sys.stdout.flush()
            
            if next_char in ['.']:
                generated_all.append(generated)
                break
        print()
            


반복번호 =  1
Epoch 1/1
15229/15229 [==============================] - 40s 3ms/step - loss: 8.7257

추출옵션 :  1.4
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지. 고함을

추출옵션 :  1.6
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지. 쟈켓을

추출옵션 :  1.8
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지. 소리:(두목의,

추출옵션 :  2.0
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지. 총에

추출옵션 :  2.2
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지. 단신으로

추출옵션 :  2.4
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지. 놈이야?

추출옵션 :  2.6
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지. 편하게

반복번호 =  2
Epoch 1/1
15229/15229 [==============================] - 43s 3ms/step - loss: 7.9575

추출옵션 :  1.4
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지. 가.

추출옵션 :  1.6
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지. 나가떨어진다.

추출옵션 :  1.8
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지. 어디

추출옵션 :  2.0
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지. 흑장갑:(목장갑의

추출옵션 :  2.2
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지. 뭐하냐?

추출옵션 :  2.4
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지. 한번만

추출옵션 :  2.6
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지. 인권유린이라는

반복번호 =  3
Epoch 1/1
15229/15229 

In [9]:
np.set_printoptions(threshold=1000000)
zz = np.zeros((1, maxlen, len(chars)))
print(zz.shape)
zz[0,0,0] = 1
zz[0,1,41] = 1
zz[0,2,23] = 1
alpha = model.predict(zz)[0]
# def sample(preds, temp=1.0):
#     preds = np.asarray(preds).astype('float64')
#     preds = np.log(preds) / temp
#     exp_preds = np.exp(preds)
#     preds = exp_preds / np.sum(exp_preds)
#     probas = np.random.multinomial(1, preds, 1)
#     return np.argmax(probas)
alpha2 = np.log(alpha)
alpha3 = np.exp(alpha2) / 1.4
alpha4 = alpha3 / np.sum(alpha3)
alpha5 = np.random.multinomial(1,alpha4, 1)
alpha6 = np.argmax(alpha5)
print(alpha4)

(1, 5, 7696)
[6.84385793e-03 2.42331716e-05 2.38502162e-05 5.09902347e-05
 1.58885305e-05 3.12381526e-05 1.51966979e-05 3.89987144e-05
 5.88389812e-05 4.55199079e-05 7.45022226e-06 1.31480219e-05
 5.23671151e-05 2.52569753e-05 2.39074197e-05 6.20746141e-05
 2.39919518e-05 2.56517051e-05 3.70215821e-05 1.35653709e-05
 1.80621591e-05 8.19426350e-05 6.31120056e-05 2.35870593e-05
 1.73951430e-05 8.28767661e-05 1.79868857e-05 1.49676880e-05
 7.04776685e-05 2.68954245e-05 2.88100346e-05 1.28401171e-05
 1.21752208e-03 9.10977978e-05 5.90078707e-04 1.93624801e-05
 1.57374270e-05 2.16998433e-05 2.08872916e-05 2.51067377e-05
 3.13659475e-05 5.48843527e-05 2.38322064e-05 2.25642252e-05
 3.59195910e-05 2.61382156e-05 3.82553917e-05 2.62243320e-05
 3.12517695e-05 5.43744136e-05 1.90239170e-05 1.66010450e-05
 1.80587649e-05 2.25209478e-05 9.90721855e-06 1.52775046e-05
 1.23454402e-05 8.16721877e-06 3.32282070e-05 2.78922107e-05
 1.87155183e-05 9.33216779e-06 3.46137058e-05 6.04892630e-05
 3.16685291

In [10]:
preds

array([4.65915131e-04, 1.89412203e-05, 7.49715109e-05, 2.57443025e-05,
       1.45906279e-05, 3.81704340e-05, 1.53089768e-05, 2.51468446e-05,
       2.44689818e-05, 2.12077393e-05, 6.35749666e-06, 2.65347408e-05,
       6.95518902e-05, 1.88917256e-05, 1.93025571e-05, 2.03398486e-05,
       3.79028024e-05, 2.39772617e-05, 3.98933553e-05, 2.53985509e-05,
       2.92690238e-05, 7.33177149e-05, 3.18420243e-05, 5.29154704e-06,
       1.04161318e-05, 6.29805654e-05, 3.39402795e-05, 2.18882378e-05,
       2.99127551e-05, 2.75400653e-05, 3.54570184e-05, 1.29106256e-05,
       1.60945492e-04, 2.93698231e-05, 1.99133196e-04, 1.95431403e-05,
       3.01035216e-05, 2.29329580e-05, 2.20144211e-05, 2.58828513e-05,
       6.50141519e-05, 1.34928578e-05, 2.42536280e-05, 3.87293294e-05,
       3.69506379e-05, 1.51730756e-05, 3.03465549e-05, 4.05945902e-05,
       6.91915266e-05, 8.10465281e-05, 2.93600206e-05, 3.03713241e-05,
       1.16049951e-05, 4.02582846e-05, 6.53862026e-06, 9.93413869e-06,
      